In [2]:
pip install cufflinks
pip install plotly==3.8.1

SyntaxError: invalid syntax (<ipython-input-2-af95afbc001e>, line 1)

In [3]:
import pandas as pd
df_i=pd.read_csv('C:/Users/user/Desktop/critical_shoot_project/01.source/01.데이터가공/02.데이터/서울지하철역좌표.csv')
df_n=pd.read_excel('C:/Users/user/Desktop/critical_shoot_project/01.source/01.데이터가공/02.데이터/지하철_선후불카드_이용객현황.xlsx')

In [4]:
df_i=df_i.drop(['Unnamed: 0'],axis=1)
df_i.rename(columns={'지하철역':'지하철역명'},inplace=True)
df_i

,지하철역명,X좌표,Y좌표
0,가락시장,127.118234,37.492522
1,종로3가,126.991806,37.571607
2,오금,127.128111,37.502162
3,동대문,127.009745,37.571420
4,동대문역사문화공원,127.007896,37.565138
...,...,...,...
584,동백,127.152716,37.269043
585,초당,127.159443,37.260752
586,삼가,127.168075,37.242115
587,시청·용인대,127.178406,37.239151


In [5]:
df_n=df_n.drop(['Unnamed: 0'],axis=1)
df_n.rename(columns={'역명':'지하철역명'},inplace=True)
df_n['이용객수']=df_n['승차총객수']+df_n['하차총승객수']
df_n2=df_n.pivot_table(index='지하철역명',columns='사용월',values='이용객수')
df_n

,사용월,노선명,지하철역명,승차총객수,하차총승객수,이용객수
0,201901,1호선,동대문,442372,481531,923903
1,201901,1호선,동묘앞,320717,336262,656979
2,201901,1호선,서울역,1737632,1612228,3349860
3,201901,1호선,시청,779320,777299,1556619
4,201901,1호선,신설동,487027,473981,961008
...,...,...,...,...,...,...
18584,202107,중앙선,원덕,9372,9191,18563
18585,202107,중앙선,중랑,142833,136184,279017
18586,202107,중앙선,지평,2016,2207,4223
18587,202107,중앙선,팔당,27522,27617,55139


In [8]:
df_ii=pd.merge(df_n2,df_i,on='지하철역명',how='left')
df_ii.dropna(inplace=True)
df_ii.reset_index()
df_iii=df_ii.sort_values(by=int(201901),ascending=False)
dff_2019=pd.concat([df_ii.iloc[:,0],df_ii.iloc[:,:13]],axis=1)
dff_2020=pd.concat([df_ii.iloc[:,0],df_ii.iloc[:,13:25]],axis=1)
dff_2021=pd.concat([df_ii.iloc[:,0],df_ii.iloc[:,25:-2]],axis=1)
df_ii['2021년평균']=dff_2021.mean(axis=1)
df_ii['2020년평균']=dff_2020.mean(axis=1)
df_ii['2019년평균']=dff_2019.mean(axis=1)

In [9]:
#월별 평균 지하철 이용객 수
df_mean=df_ii.iloc[:,:-2].mean()
df_month=pd.DataFrame(df_mean)
df_month.columns=['월별평균']
df_month_new=[]
for i in range(len(df_month.index)):
    df_month_new.append(str(df_month.index[i])[:4]+'-'+str(df_month.index[i])[4:])
df_month.index=df_month_new

In [10]:
#월별 지하철이용객 추이
import plotly.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)

df_month.iplot(kind='area')

In [16]:
#연도별 지하철이용객 추이
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rcParams['font.family'] ='Malgun Gothic'

matplotlib.rcParams['axes.unicode_minus'] =False

df_2019=df_ii.iloc[:,:13].mean()
df_2020=df_ii.iloc[:,13:25].mean()
df_2021=df_ii.iloc[:,25:-5].mean()
df_2019.mean()
df_2020.mean()
df_2021.mean()
df_year=pd.DataFrame({'연도별평균':[df_2019.mean(),df_2020.mean(),df_2021.mean()]},index=['2019년','2020년','2021년'])
df_year.iplot(kind='bar')


In [12]:
import folium.plugins
m_k = folium.plugins.DualMap(location=[37.53, 126.96], zoom_start=11)

for k in df_ii.index :
    try:
        lon=df_ii.loc[k,'X좌표']
        lat=df_ii.loc[k,'Y좌표']      
        folium.CircleMarker(
            [lat,lon],
            radius=df_ii.loc[k,'2019년평균']/100000,
            popup=df_ii.loc[k,'지하철역명'],
            color='blue',
            weight=1.5,
            fill=True).add_to(m_k.m1)
    except:
        pass


In [13]:
for k in df_ii.index :
    try:
        lon=df_ii.loc[k,'X좌표']
        lat=df_ii.loc[k,'Y좌표']      
        folium.CircleMarker(
            [lat,lon],
            radius=df_ii.loc[k,'2020년평균']/100000,
            popup=df_ii.loc[k,'지하철역명'],
            color='red',
             title='2020년평균',
            weight=1.5,
            fill=True).add_to(m_k.m2)
    except:
        pass

In [14]:
#파랑색 2019년
#빨강색 2020년
m_k